# Explore here

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
def dummy_npwarn_decorator_factory():
  def npwarn_decorator(x):
    return x
  return npwarn_decorator
np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)
import torch as to
import torch.nn as nn
import pandas as pd
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from PIL import Image


model = Sequential()
model.add(Conv2D(input_shape = (224,224,3), filters = 64, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 64,kernel_size = (3,3),padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Flatten())
model.add(Dense(units = 4096,activation = "relu"))
model.add(Dense(units = 4096,activation = "relu"))
model.add(Dense(units = 2, activation = "softmax"))

In [2]:
import torch.nn as nn
import torch

# Define a simpler CNN model using Sequential
model = nn.Sequential(
    # Block 1
    nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),  # 224 → 112

    # Block 2
    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),  # 112 → 56

    # Block 3
    nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),  # 56 → 28

    # Block 4 (Final Conv Layer)
    nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),  # 28 → 14

    # Flatten
    nn.Flatten(),

    # Fully Connected Layers (Reduced Size)
    nn.Linear(in_features=256 * 14 * 14, out_features=128),  # Reduced from 512x7x7
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 2)  # Binary Classification (Dog vs Cat)
)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Trimmed-down model is ready for training!")


Trimmed-down model is ready for training!


In [2]:
import torch
import torch.nn as nn

# Define a function to compute the correct input size for Linear layer
def get_flattened_size():
    with torch.no_grad():
        dummy_input = torch.zeros(1, 3, 224, 224)  # (batch_size=1, channels=3, height=224, width=224)
        conv_output = conv_layers(dummy_input)
        return conv_output.view(1, -1).size(1)  # Flatten and get size

# Define CNN Layers (Convolutional + MaxPooling)
conv_layers = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),  # 224 → 112

    nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),  # 112 → 56

    nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),  # 56 → 28

    nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),  # 28 → 14

    nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2)  # 14 → 7
)

# Compute the correct flattened size
flattened_size = get_flattened_size()

# Define Fully Connected Layers
fc_layers = nn.Sequential(
    nn.Linear(in_features=flattened_size, out_features=512),  # Corrected in_features
    nn.ReLU(),
    nn.Linear(512, 512),
    nn.ReLU(),
    nn.Linear(512, 2)  # 2 classes (Dog vs. Cat)
)

# Combine Everything into `Sequential`
model = nn.Sequential(
    conv_layers,
    nn.Flatten(),  # Flattens the output before FC layers
    fc_layers
)

# Move model to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Model successfully built with Sequential!")


Model successfully built with Sequential!


In [3]:
import os
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader


# Define device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images
    transforms.ToTensor(),  # Convert images to Tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize pixel values
])

# Custom Dataset Class for Ordered Cats & Dogs Images
class CatsDogsDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.image_files = sorted(
            [f for f in os.listdir(folder_path) if f.lower().endswith(('jpg', 'jpeg', 'png'))]
        )

        # First 12,500 images are cats (label=0), last 12,500 are dogs (label=1)
        self.labels = [0] * 12500 + [1] * 12500

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.folder_path, img_name)
        image = Image.open(img_path).convert("RGB")

        label = self.labels[idx]  # Assign label based on index

        if self.transform:
            image = self.transform(image)

        return image, label

# Define dataset paths
train_path = "/workspaces/classifier/dogs-vs-cats/train"
test_path = "/workspaces/classifier/dogs-vs-cats/test1"

# Create Dataset and DataLoader
train_dataset = CatsDogsDataset(train_path, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = CatsDogsDataset(test_path, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Print sample data
for images, labels in train_loader:
    print(f"Batch of images shape: {images.shape}")  # Should be (batch_size, 3, 224, 224)
    print(f"Batch of labels: {labels}")  # Should be 0 for Cats, 1 for Dogs
    break  # Print one batch and exit


Batch of images shape: torch.Size([32, 3, 224, 224])
Batch of labels: tensor([0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
        0, 0, 0, 1, 0, 1, 0, 0])


In [4]:





device = to.device("cuda" if torch.cuda.is_available() else "cpu")

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images
    transforms.ToTensor(),  # Convert images to Tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize pixel values
])

# Custom Dataset Class to Load Only 5,000 Images (2,500 Cats + 2,500 Dogs)
class CatsDogsSubsetDataset(Dataset):
    def __init__(self, folder_path, transform=None, subset_size=5000):
        self.folder_path = folder_path
        self.transform = transform
        self.image_files = sorted(
            [f for f in os.listdir(folder_path) if f.lower().endswith(('jpg', 'jpeg', 'png'))]
        )

        # Load only the first 2500 cats and 2500 dogs
        self.image_files = self.image_files[:2500] + self.image_files[12500:15000]  # 2500 Cats + 2500 Dogs

        # Assign labels: 0 for Cats, 1 for Dogs
        self.labels = [0] * 2500 + [1] * 2500  

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.folder_path, img_name)
        image = Image.open(img_path).convert("RGB")

        label = self.labels[idx]  # Assign label based on index

        if self.transform:
            image = self.transform(image)

        return image, label

# Define dataset path
train_path = "/workspaces/classifier/dogs-vs-cats/train"

# Create Dataset and DataLoader for Only 5,000 Images
train_dataset = CatsDogsSubsetDataset(train_path, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Print dataset size
print(f"Total images loaded for training: {len(train_dataset)}")  # Should print 5000



Total images loaded for training: 5000


In [5]:
# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

print("Training Complete!")


Epoch 1/10, Loss: 0.6467949376700405


KeyboardInterrupt: 